# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,10.95,87,57,0.99,NZ,1686627893
1,1,qusmuryn,52.4503,64.6311,10.53,72,100,6.78,KZ,1686628084
2,2,albany,42.6001,-73.9662,19.94,81,100,0.89,US,1686628084
3,3,bredasdorp,-34.5322,20.0403,14.19,81,84,11.43,ZA,1686627889
4,4,pasuquin,18.3328,120.6177,29.97,79,75,9.26,PH,1686628085


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df[city_data_df['Max Temp'] >=21]

# Drop any rows with null values
new_city_df.dropna()

# Display sample data
new_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,pasuquin,18.3328,120.6177,29.97,79,75,9.26,PH,1686628085
7,7,avarua,-21.2078,-159.7750,25.03,88,40,6.17,CK,1686627979
12,12,puerto ayora,-0.7393,-90.3518,26.42,93,72,2.68,EC,1686628085
15,15,high rock,26.6208,-78.2833,26.01,77,92,5.22,BS,1686628086
16,16,bayganin,48.6947,55.8742,29.85,18,97,1.60,KZ,1686628086


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = new_city_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,pasuquin,18.3328,120.6177,29.97,79,75,9.26,PH,1686628085,
7,7,avarua,-21.2078,-159.7750,25.03,88,40,6.17,CK,1686627979,
12,12,puerto ayora,-0.7393,-90.3518,26.42,93,72,2.68,EC,1686628085,
15,15,high rock,26.6208,-78.2833,26.01,77,92,5.22,BS,1686628086,
16,16,bayganin,48.6947,55.8742,29.85,18,97,1.60,KZ,1686628086,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
lat = (-90, 90)
lon = (-180, 180)
radius = 10000
limit = 20
filters = f"circle:{lon},{lat},{radius}"
bias = f"proximity:{lon},{lat}"
categories = 'accommodation.hotel', 'accomodation', 'building.holiday_house'
params = {
    'categories':categories,
    'limit':limit,
    'filter':filters,
    'bias':bias,
    'apiKey':geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Lat = row['Lat']
    lon=row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_response = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError, TypeError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pasuquin - nearest hotel: No hotel found
avarua - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
high rock - nearest hotel: No hotel found
bayganin - nearest hotel: No hotel found
chandler - nearest hotel: No hotel found
itacoatiara - nearest hotel: No hotel found
batie - nearest hotel: No hotel found
bhimunipatnam - nearest hotel: No hotel found
isangel - nearest hotel: No hotel found
kapa'a - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
san jose del guaviare - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
wenxing - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
tubarjal - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
alaghsas - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
prince george - nearest h

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
4,4,pasuquin,18.3328,120.6177,29.97,79,75,9.26,PH,1686628085,No hotel found
7,7,avarua,-21.2078,-159.7750,25.03,88,40,6.17,CK,1686627979,No hotel found
12,12,puerto ayora,-0.7393,-90.3518,26.42,93,72,2.68,EC,1686628085,No hotel found
15,15,high rock,26.6208,-78.2833,26.01,77,92,5.22,BS,1686628086,No hotel found
16,16,bayganin,48.6947,55.8742,29.85,18,97,1.60,KZ,1686628086,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
581,581,gharo,24.7424,67.5856,33.12,51,92,10.22,PK,1686628140,No hotel found
584,584,prineville,44.2999,-120.8345,25.73,38,100,7.72,US,1686628140,No hotel found
585,585,bam,29.1060,58.3570,34.19,13,0,6.06,IR,1686628140,No hotel found
587,587,mhamid,29.8200,-5.7200,25.28,26,0,1.68,MA,1686628141,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    frame_width = 700,
    frame_height = 500,
    tiles='OSM',
    size = 'Humidity',
    geo = True,
    scale = 1,
    color = "City",
)
# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)